In [72]:
# import the installed Jira library 
from jira import JIRA 
from openai import OpenAI
import time

instruction = "You are on the engineering team managing tickets. You transform brief software engineering task titles into a fully fleshed ticket that can be used in things like Jira. You should take in a short engineering task as input. You should output a completed scope for the task that could include descriptions, acceptance criteria, sub-tasks, assumptions, and any other relevant details.You output should be json of 'summary' and 'fleshedOut' only."
client = OpenAI(api_key="your api key here")

jiraUsername = "your username here"
jiraAPI = "your jira api key here"
jiraOptions = {'server': "https://<your company name>.atlassian.net"} 

jira = JIRA(options=jiraOptions, basic_auth=(jiraUsername, jiraAPI)) 

jiraProjectName = 'your project name'
jqlStr = 'project = ' + jiraProjectName

In [73]:
def makeAssistant(instruction, client):
    assistant = client.beta.assistants.create(
    name="jellyfish",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
    )
    return assistant

In [74]:
def getOutput(assistant, prompt):

    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
        content=prompt
    )
    run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
    )
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )

    time.sleep(8)
    
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    for message in reversed(messages.data):
        allData = message.content[0].text.value

    print(allData)

    summ = (allData.split("summary")[1].split('fleshedOut')[0]).replace('\n', "")

    desc = (allData.split('fleshedOut')[1]).replace('\n', "")

    return summ, desc

In [75]:
def addToJira(summ, desc, jira, jiraProjectName):
    # Create a new issue
    issue_dict = {
        'project': {'key': jiraProjectName},
        'summary': summ,
        'description': desc,
        'issuetype': {'name': 'Task'},
    }
    
    try:
        new_issue = jira.create_issue(fields=issue_dict)
        print(f"Issue created: {new_issue.key}")
    except Exception as e:
        print(f"Error: {e}")

In [76]:
assistant = makeAssistant(instruction, client)


In [77]:
summ, desc = getOutput(assistant, "Make an FAQ section")


```json
{
  "summary": "Create FAQ section for the website",
  "fleshedOut": {
    "description": "Add an FAQ section to the website to provide answers to common questions and improve user experience.",
    "acceptanceCriteria": [
      "Design and layout of the FAQ section should be consistent with the overall website theme and style.",
      "FAQ section should be easily accessible from the main navigation or prominent area of the website.",
      "Minimum of 10 frequently asked questions should be included with clear and concise answers.",
      "Each question in the FAQ section should have a toggle or expand/collapse functionality to show/hide the answer.",
      "Ensure that the FAQ section is mobile responsive for optimal viewing on all devices.",
      "Test the FAQ section across different browsers to ensure compatibility and functionality."
    ],
    "subTasks": [
      "Design the layout and user interface for the FAQ section.",
      "Gather a list of frequently asked quest

In [78]:
addToJira(summ, desc, jira, jiraProjectName)

Issue created: KAN-9


In [79]:
# Search all issues mentioned against a project name. 
for singleIssue in jira.search_issues(jql_str=jqlStr): 
    print('{}: {}:{}'.format(singleIssue.key, singleIssue.fields.summary, 
                             singleIssue.fields.reporter.displayName)) 

KAN-9: ": "Create FAQ section for the website",  ":Ananya Gupta
KAN-8: ": "Create FAQ Section",  ":Ananya Gupta
KAN-7: ": "Create an FAQ section on the website",  ":Ananya Gupta
KAN-6: ": "Create an FAQ section on the website",  ":Ananya Gupta
KAN-5: ": "New font to landing page",  ":Ananya Gupta
KAN-4: ": "Implement new font on landing page",  ":Ananya Gupta
KAN-3: New issue created from Python:Ananya Gupta
KAN-2: New issue created from Python:Ananya Gupta
KAN-1: 401 log in:Ananya Gupta
